# Download and Save the Model

To save this model so that you can use it from various locations, including other notebooks or the model server, upload it to s3-compatible storage.

## Install the required packages and define a function for the upload

In [1]:
!pip install boto3 botocore


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Download from Huggingface

In [2]:
!git config --global credential.helper store
!pip install --upgrade huggingface_hub
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


In [3]:
!git clone https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3

Cloning into 'Mistral-7B-Instruct-v0.3'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 55 (delta 20), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (55/55), 718.73 KiB | 7.19 MiB/s, done.


## Helper functions for upload

In [4]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

#upload the model directory without git
def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            if ".git" in relative_path:
                continue
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

## Check the Storage Bucket

In your S3 bucket, under the `models` upload prefix, run the `list_object` command. As best practice, to avoid mixing up model files, keep only one model and its required files in a given prefix or directory. This practice allows you to download and serve a directory with all the files that a model requires. 

If this is the first time running the code, this cell will have no output or the fraud model from the predictive AI/ML exercise.


In [5]:
list_objects("models")

## Upload and check again

Use the function to upload the `models` folder in a rescursive fashion:

In [6]:
upload_directory_to_s3("Mistral-7B-Instruct-v0.3", "models/Mistral-7B-Instruct-v0.3")

Mistral-7B-Instruct-v0.3/tokenizer_config.json -> models/Mistral-7B-Instruct-v0.3/tokenizer_config.json
Mistral-7B-Instruct-v0.3/model.safetensors.index.json -> models/Mistral-7B-Instruct-v0.3/model.safetensors.index.json
Mistral-7B-Instruct-v0.3/tokenizer.model -> models/Mistral-7B-Instruct-v0.3/tokenizer.model
Mistral-7B-Instruct-v0.3/README.md -> models/Mistral-7B-Instruct-v0.3/README.md
Mistral-7B-Instruct-v0.3/config.json -> models/Mistral-7B-Instruct-v0.3/config.json
Mistral-7B-Instruct-v0.3/special_tokens_map.json -> models/Mistral-7B-Instruct-v0.3/special_tokens_map.json
Mistral-7B-Instruct-v0.3/model-00003-of-00003.safetensors -> models/Mistral-7B-Instruct-v0.3/model-00003-of-00003.safetensors
Mistral-7B-Instruct-v0.3/model-00001-of-00003.safetensors -> models/Mistral-7B-Instruct-v0.3/model-00001-of-00003.safetensors
Mistral-7B-Instruct-v0.3/params.json -> models/Mistral-7B-Instruct-v0.3/params.json
Mistral-7B-Instruct-v0.3/tokenizer.model.v3 -> models/Mistral-7B-Instruct-v0.3

To confirm this worked, run the `list_objects` function again:

In [7]:
list_objects("models")

models/Mistral-7B-Instruct-v0.3/README.md
models/Mistral-7B-Instruct-v0.3/config.json
models/Mistral-7B-Instruct-v0.3/generation_config.json
models/Mistral-7B-Instruct-v0.3/model-00001-of-00003.safetensors
models/Mistral-7B-Instruct-v0.3/model-00002-of-00003.safetensors
models/Mistral-7B-Instruct-v0.3/model-00003-of-00003.safetensors
models/Mistral-7B-Instruct-v0.3/model.safetensors.index.json
models/Mistral-7B-Instruct-v0.3/params.json
models/Mistral-7B-Instruct-v0.3/special_tokens_map.json
models/Mistral-7B-Instruct-v0.3/tokenizer.json
models/Mistral-7B-Instruct-v0.3/tokenizer.model
models/Mistral-7B-Instruct-v0.3/tokenizer.model.v3
models/Mistral-7B-Instruct-v0.3/tokenizer_config.json
